# `astropquery.vizier` cheat sheet
This notebook is a small cheat sheet on how to query *arbitrary* astronomical catalogues from Vizier with the `astroquery` module.

Besides `astroquery`, knowledge on the modules `astropy.units`, `astropy.coordinates` and `astropy.tables` (amongst others) comes in handy. Those will be covered at some point in other cheat sheets.

Please see the end of the notebook for the versions of `python` and `python`-modules I used.

[Here](code/astroquery_vizier.py) you can find the essential code cells in the form of a `python`-script.

In [1]:
# necessary modules for this sheet:
import astropy.units as au
import astropy.coordinates as ac
import astroquery.vizier as av

Our final goal is to obtain some sources from the Gaia DR2 catalogue (Ra, Dec and G-magnitude) in a 0.5 sq. degree radius around position Ra=210.0 deg and Dec=$-10$ deg. We first want to find out which Vizier catalogue contains the Gaia DR2 data.

It should be obvious from the following cell how to proceed if you are interested in other data; e.g. just substitute `Gaia DR2` with `KiDS` if you search for KiDS data etc. 

In [2]:
# obtain a list of available Vizier catalogues with a reference to Gaia DR2
cat_list = av.Vizier.find_catalogs("Gaia DR2")
print({k:v.description for k,v in cat_list.items()})

{'I/345': 'Gaia DR2 (Gaia Collaboration, 2018)', 'I/347': 'Distances to 1.33 billion stars in Gaia DR2 (Bailer-Jones+, 2018)', 'J/A+A/616/A12': 'Gaia DR2 sources in GC and dSph (Gaia Collaboration+, 2018)', 'J/A+A/616/A37': 'Close encounters to the Sun in Gaia DR2 (Bailer-Jones+, 2018)', 'J/A+A/616/L2': 'Planetary Nebulae distances in Gaia DR2 (Kimeswenger+, 2018)'}


It is obvious that catalogue `I/345` is what we are looking for. Next, we make a basic query around our position and see what quantities the catalogue offers:

In [3]:
# define a sky-coordinate of our position and make a first query:
sky_coord = ac.SkyCoord(ra=210.0, dec=-10.0, unit=(au.deg, au.deg))

# do a first query with a radius of 0.05 deg around our position
# with 'all' quantities available.
# 
# The '**' for the columns means 'all available quantities'
gaia_query = av.Vizier(columns = ["**"])

# Note: instead of searching objects in a radius around central
#       coordinates, you also can specify width and height for a rectangular
#       region (see commented function call)
first_query = gaia_query.query_region(sky_coord, radius=0.05*au.deg, catalog='I/345')

#first_query = gaia_query.query_region(sky_coord,
#                                      width=0.05 * au.deg,
#                                      height=0.05 * au.deg,
#                                      catalog='I/345')

print(first_query)

TableList with 2 tables:
	'0:I/345/gaia2' with 102 column(s) and 37 row(s) 
	'1:I/345/ssoobs' with 27 column(s) and 2 row(s) 


The query returns results in two (sub)source lists; please see the [Vizier WEB pages](http://vizier.u-strasbg.fr/viz-bin/VizieR?-source=I/345) for what they are. We need the `I/345/gaia2` sources.

In [4]:
print(first_query['I/345/gaia2'])

   _r              DR2Name            ...     DEJ2000     e_DEJ2000
  deg                                 ...       deg          mas   
-------- ---------------------------- ... --------------- ---------
0.049630 Gaia DR2 3615329265900282496 ... -10.04731417520   23.1798
0.034431 Gaia DR2 3615329403339203072 ... -10.03399973040    5.4039
0.029292 Gaia DR2 3615329403339203456 ... -10.02881246230    9.1350
0.039968 Gaia DR2 3615329334619759872 ... -10.03966784020   22.9965
0.034307 Gaia DR2 3615329368979465472 ... -10.02508151320   12.3064
0.030759 Gaia DR2 3615329403339203200 ... -10.03057564240   10.3274
0.036726 Gaia DR2 3615329338914587392 ... -10.03497504280   22.5102
0.035201 Gaia DR2 3615330124893743488 ... -10.01415678350   19.8746
0.040700 Gaia DR2 3615330129188542976 ... -10.01927590590    1.0747
0.032439 Gaia DR2 3615330129188255488 ... -10.01400928050    6.3145
     ...                          ... ...             ...       ...
0.038466 Gaia DR2 3615424236216282112 ...  -9.97

**Note:** The result of the query is an `astropy.table` object. If you are not familiar with it, you should have a look at [this page](http://docs.astropy.org/en/stable/table/) at some point.

O.k. Let us see, what the cataogue offers and what we finally need. The following command lists all quantities with their units and a short descrption:

In [ ]:
# Note: Please run the notebook yourself to see the output of the
# following cell. It would be too big and not really useful here.
print(first_query['I/345/gaia2'].info)

Going through the output of the cell above, we identify the quantities `RAJ2000`, `DEJ2000`, `e_RAJ2000`, `e_DEJ2000`, `Gmag` and `e_Gmag`. We define a new query for a table with these columns and a radius of 0.5 sq. degrees around our coordinates.

In [5]:
# define query for the columns we want:
gaia_query = av.Vizier(columns = ["RAJ2000", "DEJ2000",
                                  "e_RAJ2000", "e_DEJ2000", "Gmag", "e_Gmag"])

# the following line is necessary that we get *all* sources and not only the first 50:
gaia_query.ROW_LIMIT = -1
query_result = gaia_query.query_region(sky_coord, radius=0.5 * au.deg,
                                       catalog='I/345')

print(query_result['I/345/gaia2'])

    RAJ2000         DEJ2000     e_RAJ2000 e_DEJ2000   Gmag  e_Gmag
      deg             deg          mas       mas      mag    mag  
--------------- --------------- --------- --------- ------- ------
210.09818004000 -10.47705412260   17.1418   12.4692 19.9721 0.0066
210.11374455600 -10.47889809570   13.8468    9.6949 19.5045 0.0049
210.10041218500 -10.47451610980    1.4436    1.2227 15.2191 0.0008
210.09935845500 -10.47421244060   12.3307    9.4673 19.0598 0.0053
210.14173754100 -10.47672446370   14.7450   10.2879 19.4335 0.0150
210.14565966800 -10.47012077380   20.5948   16.8847 20.0346 0.0088
210.15799876400 -10.47198806740   17.0740   13.0038 19.8334 0.0072
210.15671894100 -10.46717056340   13.1674    9.7731 19.5070 0.0065
210.16155320000 -10.46157060440   16.7547   12.1941 19.7505 0.0067
210.13158960300 -10.45846240970    2.9899    2.2955 20.0448 0.0138
            ...             ...       ...       ...     ...    ...
210.22911171900  -9.56777260186   20.1259   18.3684 19.7609 0.

We want to export the table to an `ASCII`-file. We note that the position errors are in units of milliarcseconds. We convert them to degrees before the export:

In [6]:
query_result['I/345/gaia2']['e_RAJ2000'] = \
  query_result['I/345/gaia2']['e_RAJ2000'].to(au.deg)
query_result['I/345/gaia2']['e_DEJ2000'] = \
  query_result['I/345/gaia2']['e_DEJ2000'].to(au.deg)
print(query_result['I/345/gaia2'])    

    RAJ2000         DEJ2000         e_RAJ2000     ...   Gmag  e_Gmag
      deg             deg              deg        ...   mag    mag  
--------------- --------------- ----------------- ... ------- ------
210.09818004000 -10.47705412260 4.76161111111e-06 ... 19.9721 0.0066
210.11374455600 -10.47889809570 3.84633333333e-06 ... 19.5045 0.0049
210.10041218500 -10.47451610980          4.01e-07 ... 15.2191 0.0008
210.09935845500 -10.47421244060 3.42519444444e-06 ... 19.0598 0.0053
210.14173754100 -10.47672446370 4.09583333333e-06 ... 19.4335 0.0150
210.14565966800 -10.47012077380 5.72077777778e-06 ... 20.0346 0.0088
210.15799876400 -10.47198806740 4.74277777778e-06 ... 19.8334 0.0072
210.15671894100 -10.46717056340 3.65761111111e-06 ... 19.5070 0.0065
210.16155320000 -10.46157060440 4.65408333333e-06 ... 19.7505 0.0067
210.13158960300 -10.45846240970 8.30527777778e-07 ... 20.0448 0.0138
            ...             ...               ... ...     ...    ...
210.22911171900  -9.56777260186 5.

Finally export the table to ASCII-catalogues. Of course you can export to any format that `astropy.table` supports - see [this page](http://docs.astropy.org/en/stable/io/unified.html#table-io) for more details.

In [ ]:
# writes an ASCII catalogue to your disk
query_result['I/345/gaia2'].write('gaia.txt', format='ascii.commented_header',
                                  overwrite=True)
# writes an 'extended csv-catalogue' to your disk. This ASCII-format also
# contains the catalogues meta-data (units and column description) in
# a standardized form.
query_result['I/345/gaia2'].write('gaia.csv', format='ascii.ecsv',
                                  overwrite=True)

## Software versions
Here are the Python and module versions I used to create this notebook

In [7]:
%reload_ext version_information
%version_information astropy, astroquery

Software versions
Python 3.6.6 64bit [GCC 7.2.0]
IPython 6.5.0
OS Linux 3.13.0 147 generic x86_64 with debian jessie sid
astropy 3.0.4
astroquery 0.3.8
Mon Aug 20 00:24:24 2018 CEST